In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction import DictVectorizer
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler


In [55]:
#读取数据
def input_data():
    train_data = pd.read_csv("./train.csv")
    test_data = pd.read_csv("./test.csv")
    test_label = pd.read_csv("./gender_submission.csv")
    test_data["Survived"] = test_label.Survived
    return train_data,test_data
train_data,test_data = input_data()
print(train_data.info())
print("=="*30)
# print(test_data.info())
print(train_data.describe(include=['O']))
# print("=="*30)
# print(test_data.describe(include=['O']))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB
None
                       Name   Sex  Ticket    Cabin Embarked
count                   891   891     891      204      889
unique                  891     2     681      147        3
top     Leonard, Mr. Lionel  male  347082  B96 B98        S
freq                      1   577       7        4      644


In [57]:
#数据处理
def data_do(dataset):
    dataset.Age.fillna(dataset.Age.mean(),inplace=True)
    dataset.Embarked.fillna('S',inplace=True)
    dataset.Fare.fillna(dataset.Fare.mean(),inplace=True)
    return dataset
train_data = data_do(train_data)
test_data = data_do(test_data)

In [58]:
#提取特征
features = ['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']
label = ['Survived']
train_features = train_data[features]
test_features = test_data[features]
train_label = train_data[label]
test_label = test_data[label]

In [59]:
#特征处理
train_featurs1 = pd.concat([pd.get_dummies(train_features.Pclass,prefix='Pclass'),train_features],axis=1).drop("Pclass",axis=1)
test_features1 = pd.concat([pd.get_dummies(test_features.Pclass,prefix='Pclass'),test_features],axis=1).drop("Pclass",axis=1)
print(test_features1.shape)
print(type(test_features1))
dvec = DictVectorizer(sparse=False)
train_features2 = dvec.fit_transform(train_featurs1.to_dict(orient='records'))
test_features2 = dvec.transform(test_features1.to_dict(orient='records'))
print(test_features2.shape)


(418, 9)
<class 'pandas.core.frame.DataFrame'>
(418, 12)


In [60]:
dvec.feature_names_

['Age',
 'Embarked=C',
 'Embarked=Q',
 'Embarked=S',
 'Fare',
 'Parch',
 'Pclass_1',
 'Pclass_2',
 'Pclass_3',
 'Sex=female',
 'Sex=male',
 'SibSp']

In [61]:
sds = StandardScaler()

train_age = sds.fit_transform(train_features2[:,0].reshape(-1,1))
test_age = sds.transform(test_features2[:,0].reshape(-1,1))

train_features3 = np.hstack((train_features2[:,1:],train_age))
test_features3 = np.hstack((test_features2[:,1:],test_age))

In [62]:
lg = LogisticRegression()
lg.fit(train_features3,train_label)

D:\conda\parse\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\conda\parse\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [63]:
lg.score(train_features3,train_label)

0.8047138047138047

In [66]:
lg.score(test_features3,test_label)

0.9473684210526315